<a href="https://colab.research.google.com/github/TruptiLomate/YBI-Foundation-Projects/blob/main/Movie_Recommendations_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

# Load the dataset
# Assuming you have a dataset with columns: 'user_id', 'movie_id', and 'rating'
# If not, use a dataset like MovieLens and adapt as needed.
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')

# Merge ratings and movies data for a complete dataset
data = pd.merge(ratings, movies, on='movie_id')

# Create a user-item matrix (pivot table with users as rows, movies as columns)
user_movie_matrix = data.pivot_table(index='user_id', columns='movie_id', values='rating')

# Fill NaN values with 0 (if needed)
user_movie_matrix.fillna(0, inplace=True)

# Standardize the matrix for better performance in similarity calculation
scaler = StandardScaler()
user_movie_matrix_std = scaler.fit_transform(user_movie_matrix)

# Calculate the Cosine Similarity
cosine_sim = cosine_similarity(user_movie_matrix_std)
cosine_sim_df = pd.DataFrame(cosine_sim, index=user_movie_matrix.index, columns=user_movie_matrix.index)

# Function to get recommendations
def recommend_movies(user_id, num_recommendations=5):
    # Get similarity scores for the given user
    user_sim_scores = cosine_sim_df[user_id].sort_values(ascending=False)

    # Get the top similar users
    top_users = user_sim_scores.iloc[1:num_recommendations+1].index

    # Collect all movies rated by the top users
    recommended_movies = user_movie_matrix.loc[top_users].mean().sort_values(ascending=False)

    # Filter out movies the user has already rated
    user_rated_movies = user_movie_matrix.loc[user_id]
    recommended_movies = recommended_movies[user_rated_movies.isna()]

    # Get top N recommendations
    return recommended_movies.head(num_recommendations)

# Test the function for a specific user
user_id = 1  # Change this to test for different users
recommendations = recommend_movies(user_id)
print("Recommended Movies for User", user_id)
print(recommendations)